In [1]:
import csv
import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

Using TensorFlow backend.


TensorFlow version:  2.1.0


In [2]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import average_precision_score

In [4]:
# Define paths
base_path = os.path.abspath("./")
dataset_path = os.path.join(base_path, "dataset", "isic2016numpy")
model_path = os.path.join(base_path, "models")
print(os.listdir(dataset_path))

['x_test.npy', 'x_testFeatures.npy', 'x_train.npy', 'x_trainFeatures.npy', 'y_test.npy', 'y_train.npy']


In [5]:
# Load data
x_train = np.load("{}/x_trainFeatures.npy".format(dataset_path)) 
y_train = np.load("{}/y_train.npy".format(dataset_path))
x_test = np.load("{}/x_testFeatures.npy".format(dataset_path))
y_test = np.load("{}/y_test.npy".format(dataset_path))

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
def construct_network(dropout, neuronPct, neuronShrink):
    neuronCount = int(neuronPct * 5000)
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Flatten(input_shape=x_train.shape[1:]))
    #model.add(Input(shape=(1,512)))
    #model.add(Flatten())
    model.add(Dense(neuronCount))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation='sigmoid')) # Output
    return model

In [8]:
x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255


In [9]:
# Define class weights for imbalacned data
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(np.argmax(y_train, axis=1)), np.argmax(y_train, axis=1))
print(class_weights)

[0.61898212 2.60115607]


C:\ProgramData\Anaconda3\envs\keras_tuner_ori\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass classes=[0 1], y=[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [10]:
class_weights = [0.1520195864872159, 0.8479804135127841]

In [11]:
class_weights 

[0.1520195864872159, 0.8479804135127841]

In [12]:
# Train the awesome model
# Configuration
batch_size = 2
epochs = 10

In [13]:
def optimize_network(dropout,lr,neuronPct,neuronShrink,alpha,gamma):
    output_dir = os.path.join(base_path, "weights")
    output_weights_name='weights.h5'
    start_time = time.time()
    model = construct_network(dropout, neuronPct, neuronShrink)
    optimizer = SGD(lr=lr)
    model.compile(optimizer=optimizer,loss=focal_loss(gamma=gamma,alpha=alpha))
    model.summary()
    EXP_NAME = str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)
    output_path = os.path.join(output_dir,EXP_NAME+"_"+output_weights_name)

    
    checkpoint = ModelCheckpoint(
                 output_path,
                 monitor='val_loss',
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
    )
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-8, mode='auto') # new_lr = lr * factor
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, patience=8, mode='auto', restore_best_weights=True)
    csv_logger = CSVLogger('{}\{}_training.csv'.format(output_dir, EXP_NAME))
    
    model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                class_weight = class_weights,
                callbacks=[csv_logger, early_stopping, reduce_lr, checkpoint], # early_stopping, checkpointer, reduce_lr
                shuffle=False)
    
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_flat = np.argmax(y_test, axis=1)
    
    y_pred_flat = np.array([np.argmax(pred) for pred in y_pred])
    
    acc = accuracy_score(y_test_flat, y_pred_flat) * 100
    print("Accuracy :", acc)
    
    auroc = roc_auc_score(y_test, y_pred) *100
    print("Auroc :", auroc)
    
    ap = average_precision_score(y_test, y_pred) * 100
    print("Average precision :", ap)
    
    
    cm = confusion_matrix(y_pred=y_pred_flat, y_true=y_test_flat)
    total=sum(sum(cm))

    sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
    print('Sensitivity : ', sensitivity*100 )

    Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
    print('Specificity : ', Specificity*100)
    
    if str(sensitivity) == "nan":
        sensitivity = 0
    
    return sensitivity

In [14]:
import csv
import re
pbounds = {}
for key, val in csv.reader(open("new_params.csv")):
    a,b = val.split(", ")
    a = re.sub('[()]', '', a)
    b = re.sub('[()]', '', b)
    print(a)
    print(b)
    pbounds[str(key)] = float(a),float(b)

0.42102554685039006
0.4710255468503901
0.09028410678765617
0.14028410678765618
0.8564547502853013
0.9064547502853013
0.014861554283798768
0.03986155428379877
0.9523507517808921
1.002350751780892
0.2944107131397793
0.34441071313977933


In [15]:
pbounds

{'alpha': (0.42102554685039006, 0.4710255468503901),
 'dropout': (0.09028410678765617, 0.14028410678765618),
 'gamma': (0.8564547502853013, 0.9064547502853013),
 'lr': (0.014861554283798768, 0.03986155428379877),
 'neuronPct': (0.9523507517808921, 1.002350751780892),
 'neuronShrink': (0.2944107131397793, 0.34441071313977933)}

In [16]:
from new_bayes_opt.bayesian_optimization import BayesianOptimization
import time

optimizer = BayesianOptimization(
    f=optimize_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

Kunci ['alpha', 'dropout', 'gamma', 'lr', 'neuronPct', 'neuronShrink']
Bound  [[0.42102555 0.47102555]
 [0.09028411 0.14028411]
 [0.85645475 0.90645475]
 [0.01486155 0.03986155]
 [0.95235075 1.00235075]
 [0.29441071 0.34441071]]


In [17]:
from bayes_opt.util import load_logs#from bayes_opt.util import load_logsm

#load_logs(new_optimizer, logs=["./noise10_1.json"])
#print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))

load_logs(optimizer, logs=["./1.json"])
print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
new_logger = JSONLogger(path="./2.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, new_logger)

Konten (0.8923418089348906, 0.15804867401632372, 0.10021731215295529, 0.010813995435791039, 0.45136194183306017, 0.21846771895375958)
Konten (0.8923418089348906, 0.15804867401632372, 0.10021731215295529, 0.010813995435791039, 0.45136194183306017, 0.21846771895375958)
Konten (0.45389440161757466, 0.10183410905645718, 0.8538582010382728, 0.012232900404020142, 0.94583804364198, 0.3370439000793519)
Konten (0.45389440161757466, 0.10183410905645718, 0.8538582010382728, 0.012232900404020142, 0.94583804364198, 0.3370439000793519)
Konten (0.48845927448988313, 0.18171761545864185, 0.15203642707605972, 0.013022805061070413, 0.9424082162963354, 0.31173796568915035)
Konten (0.48845927448988313, 0.18171761545864185, 0.15203642707605972, 0.013022805061070413, 0.9424082162963354, 0.31173796568915035)
Konten (0.3667351833309441, 0.07971522336273182, 1.6214146804835197, 0.014809569454316385, 0.7538648833590258, 0.3384645231338628)
Konten (0.3667351833309441, 0.07971522336273182, 1.6214146804835197, 0.01

In [18]:
start_time = time.time()
optimizer.maximize(init_points=5,acq="weightedei", n_iter=20, omega=0.1)
time_took = time.time() - start_time

Isine x [0.44187665 0.12630033 0.85646047 0.02241987 0.95968855 0.29902764]
Konten (0.44187664708551877, 0.12630033145976408, 0.8564604690261686, 0.022419868599594762, 0.9596885463217477, 0.2990276428782192)
Masuk
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1434)              735642    
_________________________________________________________________
activation (Activation)      (None, 1434)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1434)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2870      
Total params: 738,512
Trainable params: 

Epoch 5/10
870/900 [============================>.] - ETA: 0s - loss: 0.4516
Epoch 00005: val_loss improved from 0.48451 to 0.48260, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10756214313980855_0.028331972633882693__0.9733104775010568_0.3286716881596173_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4492 - val_loss: 0.4826
Epoch 6/10
880/900 [============================>.] - ETA: 0s - loss: 0.4463
Epoch 00006: val_loss improved from 0.48260 to 0.48014, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10756214313980855_0.028331972633882693__0.9733104775010568_0.3286716881596173_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4441 - val_loss: 0.4801
Epoch 7/10
868/900 [===========================>..] - ETA: 0s - loss: 0.4418
Epoch 00007: val_loss improved from 0.48014 to 0.47765, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10756214313980855_0.02833197263388

Epoch 1/10
880/900 [============================>.] - ETA: 0s - loss: 0.4967
Epoch 00001: val_loss improved from inf to 0.49487, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10018918124190011_0.03906809367678371__0.9680219606888543_0.329026843923245_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4944 - val_loss: 0.4949
Epoch 2/10
874/900 [============================>.] - ETA: 0s - loss: 0.4627
Epoch 00002: val_loss improved from 0.49487 to 0.48248, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10018918124190011_0.03906809367678371__0.9680219606888543_0.329026843923245_weights.h5
900/900 [==============================] - 1s 958us/sample - loss: 0.4618 - val_loss: 0.4825
Epoch 3/10
884/900 [============================>.] - ETA: 0s - loss: 0.4473
Epoch 00003: val_loss improved from 0.48248 to 0.47846, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10018918124190011_0.03906809367678371__0

862/900 [===========================>..] - ETA: 0s - loss: 0.4642
Epoch 00008: val_loss improved from 0.49051 to 0.48844, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.13501443996284854_0.015837923864620827__0.9608422727591206_0.33831783831125_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4617 - val_loss: 0.4884
Epoch 9/10
858/900 [===========================>..] - ETA: 0s - loss: 0.4616
Epoch 00009: val_loss improved from 0.48844 to 0.48678, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.13501443996284854_0.015837923864620827__0.9608422727591206_0.33831783831125_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4579 - val_loss: 0.4868
Epoch 10/10
866/900 [===========================>..] - ETA: 0s - loss: 0.4567
Epoch 00010: val_loss improved from 0.48678 to 0.48522, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.13501443996284854_0.015837923864620827__0.960842

Train on 900 samples, validate on 379 samples
Epoch 1/10
850/900 [===========================>..] - ETA: 0s - loss: 0.5147
Epoch 00001: val_loss improved from inf to 0.51003, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10620474815350799_0.026071300692021696__0.9609610120418595_0.3095574309654645_weights.h5
900/900 [==============================] - 1s 2ms/sample - loss: 0.5120 - val_loss: 0.5100
Epoch 2/10
834/900 [==========================>...] - ETA: 0s - loss: 0.4878
Epoch 00002: val_loss improved from 0.51003 to 0.49539, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.10620474815350799_0.026071300692021696__0.9609610120418595_0.3095574309654645_weights.h5
900/900 [==============================] - 1s 993us/sample - loss: 0.4826 - val_loss: 0.4954
Epoch 3/10
860/900 [===========================>..] - ETA: 0s - loss: 0.4695
Epoch 00003: val_loss improved from 0.49539 to 0.48753, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\

...

Epoch 2/10
898/900 [============================>.] - ETA: 0s - loss: 0.4954
Epoch 00002: val_loss improved from 0.51565 to 0.50345, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.14028410678765618_0.014861554283798768__0.9633494480344686_0.2944107131397793_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4949 - val_loss: 0.5035
Epoch 3/10
872/900 [============================>.] - ETA: 0s - loss: 0.4816
Epoch 00003: val_loss improved from 0.50345 to 0.49403, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.14028410678765618_0.014861554283798768__0.9633494480344686_0.2944107131397793_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4809 - val_loss: 0.4940
Epoch 4/10
886/900 [============================>.] - ETA: 0s - loss: 0.4732
Epoch 00004: val_loss improved from 0.49403 to 0.48722, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.14028410678765618_0.01486155428379

862/900 [===========================>..] - ETA: 0s - loss: 0.4439
Epoch 00009: val_loss improved from 0.47145 to 0.46969, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1285860837791925_0.01804044234975034__1.002350751780892_0.3157567949331405_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4415 - val_loss: 0.4697
Epoch 10/10
850/900 [===========================>..] - ETA: 0s - loss: 0.4417
Epoch 00010: val_loss improved from 0.46969 to 0.46813, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.1285860837791925_0.01804044234975034__1.002350751780892_0.3157567949331405_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4377 - val_loss: 0.4681
Accuracy : 81.53034300791556
Auroc : 74.99342105263158
Average precision : 69.60316640795176
Sensitivity :  81.45161290322581
Specificity :  85.71428571428571
Konten (0.42102554685039006, 0.1285860837791925, 0.8856893371396144, 0.01804044234975034, 1.0

Epoch 2/10
868/900 [===========================>..] - ETA: 0s - loss: 0.4774
Epoch 00002: val_loss improved from 0.50792 to 0.49489, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12178456799742968_0.03986155428379877__0.992441615260079_0.2948841747361765_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4748 - val_loss: 0.4949
Epoch 3/10
860/900 [===========================>..] - ETA: 0s - loss: 0.4610
Epoch 00003: val_loss improved from 0.49489 to 0.49196, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12178456799742968_0.03986155428379877__0.992441615260079_0.2948841747361765_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4586 - val_loss: 0.4920
Epoch 4/10
844/900 [===========================>..] - ETA: 0s - loss: 0.4534
Epoch 00004: val_loss improved from 0.49196 to 0.48973, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12178456799742968_0.03986155428379877_

862/900 [===========================>..] - ETA: 0s - loss: 0.4449
Epoch 00009: val_loss improved from 0.47412 to 0.47263, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.13996809232805518_0.018344896911736422__0.9671352324053024_0.3002362375716926_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4428 - val_loss: 0.4726
Epoch 10/10
862/900 [===========================>..] - ETA: 0s - loss: 0.4408
Epoch 00010: val_loss improved from 0.47263 to 0.47124, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.13996809232805518_0.018344896911736422__0.9671352324053024_0.3002362375716926_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4386 - val_loss: 0.4712
Accuracy : 81.53034300791556
Auroc : 74.99780701754386
Average precision : 69.95033071857438
Sensitivity :  81.45161290322581
Specificity :  85.71428571428571
Konten (0.42565220112913843, 0.13996809232805518, 0.8795911351905223, 0.018344896911736

Epoch 2/10
852/900 [===========================>..] - ETA: 0s - loss: 0.4676
Epoch 00002: val_loss improved from 0.49815 to 0.48545, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12693978179249527_0.03963201123998593__0.9950761454573266_0.3068982021451878_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4650 - val_loss: 0.4855
Epoch 3/10
866/900 [===========================>..] - ETA: 0s - loss: 0.4521
Epoch 00003: val_loss improved from 0.48545 to 0.48176, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12693978179249527_0.03963201123998593__0.9950761454573266_0.3068982021451878_weights.h5
900/900 [==============================] - 1s 1ms/sample - loss: 0.4500 - val_loss: 0.4818
Epoch 4/10
882/900 [============================>.] - ETA: 0s - loss: 0.4411
Epoch 00004: val_loss improved from 0.48176 to 0.47889, saving model to C:\Users\bayuo\Desktop\skinCh4Rev\aktualV5\weights\0.12693978179249527_0.0396320112399859

In [19]:
def convert_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [20]:
print(f"Total runtime: {convert_string(time_took)}")
print(optimizer.max)

Total runtime: 0:04:20.29
{'target': 0.8194070080862533, 'params': {'alpha': 0.42102554685039006, 'dropout': 0.1192576229363119, 'gamma': 0.8832757717365153, 'lr': 0.03986155428379877, 'neuronPct': 0.9821836715963856, 'neuronShrink': 0.2944107131397793}}


In [21]:
import gc
gc.collect()

23912